In [1]:
import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
import pandas as pd
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
import yaml
import getpass

In [2]:
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument("--start-maximized")
# options.add_argument('--window-size=800x800')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('log-level=3')

config = yaml.safe_load(open('./config.yml'))

In [3]:
driver = webdriver.Chrome(config['chromedriver'], chrome_options=options)

In [4]:
url = 'https://www.linkedin.com/company/balaji-telefilms-ltd./people/'
driver.get(url)

inputElement = driver.find_element_by_id("username")
inputElement.send_keys('bhaskar2choudhary@gmail.com')

inputElement = driver.find_element_by_id("password")
inputElement.send_keys('password')

inputElement.send_keys(Keys.ENTER)

In [5]:
company_name = driver.find_element_by_css_selector('.org-top-card-summary__title.t-24.t-black.truncate').text

In [7]:
total_employees = int(driver.find_element_by_css_selector('.t-20.t-black').text.split()[0].replace(',', ''))
print('Total employees -', total_employees)

Total employees - 1119


In [8]:
people = []
while len(people) < total_employees:
    body = driver.find_element_by_css_selector('body')
    body.send_keys(Keys.END)
    time.sleep(2)
    body.send_keys(Keys.PAGE_UP)

    people = driver.find_elements_by_class_name('org-people-profiles-module__profile-item')
print('Finished getting total employees...')
print('Employees found -', len(people))

KeyboardInterrupt: 

In [11]:
people_data = []
for person in people:
    text = person.text
    name = text.split('\n')[0]
    if name != 'LinkedIn Member':
        tagline = person.find_elements_by_css_selector('.lt-line-clamp.lt-line-clamp--multi-line.ember-view')
        if tagline:
            tagline = tagline[0].text
            print(tagline)
        
        connection = person.find_elements_by_class_name('artdeco-entity-lockup__degree')
        if connection:
            connection = connection[0].text
        
        links = person.find_elements_by_css_selector('.link-without-visited-state.ember-view')
        links = [item.get_attribute('href') for item in links] if links else [None]
        
        people_data.append({
            'company': company_name,
            'name': name,
            'tagline': tagline,
            'connection': connection,
            'link': links[0]
        })

df = pd.DataFrame(people_data)

try:
    df_main = pd.read_csv('./linkedin_contacts.csv')
except:
    df_main = pd.DataFrame()

df_main = pd.concat([df_main, df], axis=0)
df_main.drop_duplicates('link', inplace=True)
df_main.to_csv('./linkedin_contacts.csv', index=False)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=80.0.3987.122)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Windows NT 10.0.18362 x86_64)
